# Setup Geral

Se estiver executando este exercício no Google Colab, execute as próximas duas células. 

Caso esteja executando localmente, não é necessário executar mas certifique-se de que o **pyspark** está instalado e configurado em sua máquina.

In [ ]:
%%bash

# Instal Java
apt-get update && apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install PySpark
pip install -q pyspark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [2,235 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:14 http://arc

In [ ]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

# Teste

O teste a ser realizado é composto de 3 partes:

- um exercício de programação em Python
- alguns exercícios de programação em SQL
- alguns exercícios de programação com PySpark

Você pode escolher qual das partes do exercício vai fazer primeiro. Todo o exercício deve ser completado no período de 48 horas.

# Python

In [ ]:
# SETUP
nomes_alunos = [
    ('Maria', 1),
    ('João', 2),
    ('Pedro', 3),
    ('Gabriella', 4),
    ('Giovana', 5),
    ('Arthur', 6)
]

notas_alunos = {
    1: 9.5,
    2: 5.1,
    3: 8.7,
    4: 7.1,
    5: 4.8,
    6: 6.3
}

Implemente uma função que recebe uma lista de nomes de alunos, um dicionário de notas dos mesmo, sendo que essas estruturas se relacionam por um ID.

A função deve retornar em ordem alfabética, o nome dos alunos que obtiveram notas maior ou igual de uma nota de corte informada.

In [ ]:
#chave de lista iniciam em 0
nomes_alunos[0]

('Maria', 1)

In [ ]:
#primeiro colchete index da lista, segundo colchete as posições dentro do index
nomes_alunos[0][0]

'Maria'

In [ ]:
#chave de discionario iniciam em 1 - variavel 
notas_alunos[1]

9.5

In [ ]:
nomes_ordenados = []
def filtra_alunos_acima_corte(alunos, notas, nota_corte):
  n = 0
  j = 1
  k = 0
  
  while k < 6:
    if notas_alunos[j] > 6:
      nomes_ordenados.append(nomes_alunos[n][0])
    n = n + 1
    j = j +1
    k = k + 1
  print(sorted(nomes_ordenados))   
filtra_alunos_acima_corte(nomes_alunos, notas_alunos, 6)


['Arthur', 'Gabriella', 'Maria', 'Pedro']


# SQL

**Setup**


In [ ]:
%%bash
mkdir bases_teste
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/produtos.csv -o bases_teste/produtos.csv
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/vendas.csv -o bases_teste/vendas.csv
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/usuarios.csv -o bases_teste/usuarios.csv

In [ ]:
# Setup Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AtividadeSQL").getOrCreate()

In [ ]:
def cria_tabela(path, nome_tabela):
    df = spark.read.csv(path, inferSchema=True, header=True)
    df.createOrReplaceTempView(nome_tabela)
    return df

usuarios = cria_tabela("bases_teste/usuarios.csv", "usuarios")
produtos = cria_tabela("bases_teste/produtos.csv", "produtos")
vendas = cria_tabela("bases_teste/vendas.csv", "vendas")

**Função para execução de queries**

In [ ]:
def q(query, n=50):
    return spark.sql(query).show(n=n, truncate=False)

Para executar alguma consulta, basta colocar seu código sql dentro da função q como no exemplo abaixo:

```python
q("""
    SELECT *
    FROM usuarios
""")
```

e o resultado será exibido na tela.

---

Nesta parte da atividade, você vai trabalhar com três tabelas:

- produtos
- usuarios
- vendas

Use-as para responder às perguntas a seguir.

1) Qual foi a quantidade de vendas nos estados de Minas Gerais e São Paulo para cada ano e mês?

In [ ]:
q("""
    DESCRIBE usuarios
""")

+-------------+---------+-------+
|col_name     |data_type|comment|
+-------------+---------+-------+
|cod_usuario  |int      |null   |
|data_cadastro|timestamp|null   |
|faixa_etaria |string   |null   |
|cidade       |string   |null   |
|estado       |string   |null   |
+-------------+---------+-------+



In [ ]:
#aumentar o número de linhas para exibição completa da querie
q("""
    SELECT count(v.cod_usuario) as vendas, YEAR(v.data_compra) ano, MONTH(v.data_compra) mes, u.estado
    FROM vendas v
    INNER JOIN usuarios u
    ON v.cod_usuario = u.cod_usuario
    WHERE estado = 'São Paulo' or estado ='Minas Gerais'
    GROUP BY YEAR(v.data_compra), MONTH(v.data_compra), u.estado
    ORDER BY 2,3,4
""")

+------+----+---+------------+
|vendas|ano |mes|estado      |
+------+----+---+------------+
|1     |2018|6  |Minas Gerais|
|1     |2018|7  |São Paulo   |
|1     |2018|8  |Minas Gerais|
|1     |2018|10 |Minas Gerais|
|1     |2018|10 |São Paulo   |
|1     |2018|11 |Minas Gerais|
|2     |2018|12 |Minas Gerais|
|2     |2019|1  |Minas Gerais|
|1     |2019|2  |Minas Gerais|
|2     |2019|4  |Minas Gerais|
|1     |2019|5  |Minas Gerais|
|1     |2019|6  |Minas Gerais|
|1     |2019|6  |São Paulo   |
|2     |2019|7  |Minas Gerais|
|1     |2019|8  |Minas Gerais|
|1     |2019|8  |São Paulo   |
|2     |2019|9  |Minas Gerais|
|1     |2019|10 |Minas Gerais|
|1     |2019|11 |Minas Gerais|
|3     |2019|12 |Minas Gerais|
|1     |2020|1  |Minas Gerais|
|1     |2020|1  |São Paulo   |
|1     |2020|2  |Minas Gerais|
|1     |2020|3  |Minas Gerais|
|1     |2020|3  |São Paulo   |
|1     |2020|4  |Minas Gerais|
|1     |2020|4  |São Paulo   |
|3     |2020|5  |Minas Gerais|
|3     |2020|6  |Minas Gerais|
|1     |

2) Quais são os usuários por Estado que mais compraram em todo o período analisado e qual foi o número de compras realizadas, a quantidade total de itens comprados e valor total pago por usuário?

In [ ]:
#quantidade de compras por usuário, quantidade de itens comprados, valor total

q("""
    SELECT u.estado, v.cod_usuario, COUNT(v.cod_usuario) qtd_compras, SUM(v.quantidade) qtd_itens, ROUND(SUM(v.valor),2) valor_total
    FROM vendas v
    INNER JOIN usuarios u
    ON v.cod_usuario = u.cod_usuario
    GROUP BY  u.estado, v.cod_usuario
    ORDER BY 1,2
""")

+------------------+-----------+-----------+---------+-----------+
|estado            |cod_usuario|qtd_compras|qtd_itens|valor_total|
+------------------+-----------+-----------+---------+-----------+
|Amapá             |2186       |7          |61       |51359.34   |
|Bahia             |176        |5          |43       |8433.46    |
|Bahia             |466        |4          |33       |13613.89   |
|Bahia             |2155       |4          |39       |13820.4    |
|Bahia             |2856       |7          |71       |71986.05   |
|Bahia             |2981       |9          |70       |126282.4   |
|Bahia             |3400       |5          |35       |7052.29    |
|Ceará             |1787       |8          |79       |121494.67  |
|Goiás             |700        |9          |74       |142389.09  |
|Goiás             |1878       |4          |54       |96862.1    |
|Goiás             |2850       |5          |64       |107155.5   |
|Goiás             |3358       |7          |58       |136881.1

3) Quais são os usuários que não fizeram nenhuma compra?

In [ ]:
# Utilizei o inner join para cruzar as tabelas e condição WHRE ISNULL para localizar os itens com valores nulos na tabela de vendas.
q("""
    SELECT u.cod_usuario
    FROM usuarios u
    LEFT JOIN vendas v
    ON v.cod_usuario = u.cod_usuario
    WHERE ISNULL(v.cod_usuario)
     
""")

+-----------+
|cod_usuario|
+-----------+
|5098       |
|5229       |
|5482       |
|5504       |
|5614       |
|5923       |
|6985       |
|7901       |
|9980       |
|10080      |
|10152      |
|10170      |
|10929      |
|11002      |
|11546      |
|12298      |
|12354      |
|13117      |
|13476      |
|14486      |
|15198      |
|15313      |
|15784      |
|16570      |
|17530      |
|17779      |
|18028      |
|18034      |
|18166      |
|18366      |
+-----------+
only showing top 30 rows



4) Qual é o ticket médio (média de valor gasto) e o número total de usuários que fizeram pelo menos uma compra por faixa etária?

In [ ]:
q("""
    SELECT *
    FROM vendas
     
""")

+-----------+-----------+-------------------+----------+--------+
|cod_usuario|cod_produto|data_compra        |quantidade|valor   |
+-----------+-----------+-------------------+----------+--------+
|927        |10         |2020-10-19 00:00:00|4         |2799.6  |
|1544       |10         |2021-03-28 00:00:00|2         |1399.8  |
|2833       |2          |2020-09-13 00:00:00|20        |26915.6 |
|2253       |14         |2021-06-21 00:00:00|1         |138.99  |
|1724       |10         |2021-07-10 00:00:00|6         |4199.4  |
|1534       |5          |2019-10-17 00:00:00|19        |8303.0  |
|1977       |14         |2020-05-08 00:00:00|2         |277.98  |
|1916       |3          |2020-12-01 00:00:00|14        |28693.0 |
|2446       |1          |2021-06-02 00:00:00|17        |78045.3 |
|2041       |4          |2021-07-25 00:00:00|19        |1119.1  |
|1731       |11         |2021-05-27 00:00:00|5         |249.5   |
|3076       |12         |2021-04-29 00:00:00|11        |852.5   |
|1514     

In [ ]:
# Utilizei o inner join para cruzar as tabelas e localizar os itens com valores nulos na tabela de vendas.

q("""
    SELECT ROUND(AVG(v.valor),2) tct_medio, COUNT(v.cod_usuario) usuarios, u.faixa_etaria
    FROM vendas v
    INNER JOIN usuarios u
    ON v.cod_usuario = u.cod_usuario
    GROUP BY u.faixa_etaria
    HAVING COUNT(v.cod_usuario) >= 1
    ORDER BY 3
     
""")

+---------+--------+------------------+
|tct_medio|usuarios|faixa_etaria      |
+---------+--------+------------------+
|8890.13  |43      |Entre 10 a 15 anos|
|12725.95 |50      |Entre 16 a 21 anos|
|12203.02 |64      |Entre 22 a 27 anos|
|8233.04  |56      |Entre 28 a 36 anos|
|16727.19 |42      |Entre 37 a 49 anos|
|10849.48 |80      |Entre 50 a 61 anos|
|10012.63 |39      |Entre 62 a 70 anos|
|11303.1  |29      |Mais de 70 anos   |
+---------+--------+------------------+



In [ ]:
spark.stop()

# PySpark

**setup**:

In [ ]:
%%bash
mkdir bases_teste
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/produtos.csv -o bases_teste/produtos.csv
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/vendas.csv -o bases_teste/vendas.csv
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/usuarios.csv -o bases_teste/usuarios.csv

In [ ]:
# Setup Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AtividadeSQL").getOrCreate()

In [ ]:
def cria_tabela(path, nome_tabela):
    df = spark.read.csv(path, inferSchema=True, header=True)
    df.createOrReplaceTempView(nome_tabela)
    return df

usuarios = cria_tabela("bases_teste/usuarios.csv", "usuarios")
produtos = cria_tabela("bases_teste/produtos.csv", "produtos")
vendas = cria_tabela("bases_teste/vendas.csv", "vendas")

In [ ]:
#Visualizar tabelas dentro spark
print(spark.catalog.listTables())

[Table(name='produtos', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='usuarios', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='vendas', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


Responda às perguntas a seguir utilizando **Spark DATAFRAMES**.

1) Qual foi o total de compras realizadas, o total de itens comprados e a receita total obtida em todo o período analisado?

In [ ]:
vendas.printSchema()

In [ ]:
#total de compras
qtd_vendas = spark.sql("""SELECT COUNT(*) qtd_vendas 
                          FROM vendas""")
qtd_vendas.show()

+----------+
|qtd_vendas|
+----------+
|     20000|
+----------+



In [ ]:
#total de itens comprados
qtd_itens = spark.sql("""SELECT SUM(quantidade) qtd_itens 
                         FROM vendas""")
qtd_itens.show()

+---------+
|qtd_itens|
+---------+
|   209149|
+---------+



In [ ]:
#receita total - 
receita = spark.sql("""SELECT SUM(valor) 
                       FROM vendas""")
receita.show()

+--------------------+
|          sum(valor)|
+--------------------+
|2.1584900165999958E8|
+--------------------+



2) Quais são os 3 produtos mais comprados dos estados da região Sul e Sudeste, a quantidade de itens comprados, o valor total pago e a média de preço paga?

In [ ]:
top_regiao3 = spark.sql("""SELECT p.nome_produto, SUM(v.quantidade) qtd, ROUND(SUM(v.valor),2) vl_total, ROUND(AVG(v.valor/v.quantidade),2) vl_medio, u.estado 
                           FROM usuarios u 
                           INNER JOIN vendas v 
                           ON v.cod_usuario = u.cod_usuario	
                           INNER JOIN produtos p 
                           ON p.cod_produto = v.cod_produto 
                           WHERE u.estado = 'São Paulo' or u.estado = 'Minas Gerais' or u.estado = 'Rio de Janeiro' or u.estado = 'Espírito Santo' or u.estado = 'Paraná' or u.estado = 'Santa Catarina' or u.estado = 'Rio Grande do Sul' 
                           GROUP BY p.nome_produto, u.estado 
                           ORDER BY 2 DESC LIMIT 3""")
top_regiao3.show()

3) Para cada produto, quantos usuários fizeram pelo menos uma compra desse produto e qual é o valor mínimo e máximo pago por eles?

In [ ]:
vendas_produtos = spark.sql("""SELECT p.nome_produto produto, COUNT(DISTINCT(v.cod_usuario)) qtd_users, ROUND(MAX(v.valor/v.quantidade),2) vl_max, ROUND(MIN(v.valor/v.quantidade),2) vl_min 
                               FROM produtos p 
                               INNER JOIN vendas v 
                               ON p.cod_produto = v.cod_produto 
                               GROUP BY 1""")
vendas_produtos.show()

4) Aplique um desconto de 10% em todas as vendas dos usuários que fizeram pelo menos 3 compras de produtos na mesma categoria, a partir da 4ª compra realizada. Exiba apenas os usuários que terão o desconto aplicado, mantendo todas as compras, o valor original e o valor com o desconto aplicado.

In [ ]:
vendas.show(1)
produtos.show(1)
usuarios.show(1)

+-----------+-----------+-------------------+----------+------+
|cod_usuario|cod_produto|        data_compra|quantidade| valor|
+-----------+-----------+-------------------+----------+------+
|        927|         10|2020-10-19 00:00:00|         4|2799.6|
+-----------+-----------+-------------------+----------+------+
only showing top 1 row

+-----------+--------------------+-----------------+-------------+
|cod_produto|        nome_produto|categoria_produto|valor_produto|
+-----------+--------------------+-----------------+-------------+
|          1|Notebook Asus Int...|       Tecnologia|       4590.9|
+-----------+--------------------+-----------------+-------------+
only showing top 1 row

+-----------+-------------------+------------------+---------+------------+
|cod_usuario|      data_cadastro|      faixa_etaria|   cidade|      estado|
+-----------+-------------------+------------------+---------+------------+
|         69|2020-07-24 00:00:00|Entre 16 a 21 anos|São Romão|Minas G

In [ ]:
desconto_vendas = spark.sql("""SELECT v.cod_usuario, COUNT(v.cod_usuario) qtd_compras, ROUND(SUM(v.valor),2) vl_total,
                               ROUND(SUM(v.valor * 0.1),2) vl_desconto,  ROUND(SUM(v.valor) - SUM(v.valor * 0.1),2) vl_liquido
                               FROM vendas v
                               INNER JOIN produtos p
                               ON p.cod_produto = v.cod_produto
                               GROUP BY v.cod_usuario
                               HAVING COUNT(v.cod_usuario) >= 4 and COUNT(v.quantidade) >=3

                              
""")
desconto_vendas.show(10)

+-----------+-----------+---------+-----------+----------+
|cod_usuario|qtd_compras| vl_total|vl_desconto|vl_liquido|
+-----------+-----------+---------+-----------+----------+
|       2366|          4|  13642.9|    1364.29|  12278.61|
|       1959|          8|187093.42|   18709.34| 168384.08|
|        463|          7| 14536.84|    1453.68|  13083.16|
|       2142|          5|155885.78|   15588.58|  140297.2|
|       1088|         11|128345.71|   12834.57| 115511.14|
|       2659|          7| 89150.46|    8915.05|  80235.41|
|       2866|         10| 70794.48|    7079.45|  63715.03|
|        471|          4| 66282.83|    6628.28|  59654.55|
|       1591|          9| 41136.15|    4113.62|  37022.53|
|        833|          7| 93297.52|    9329.75|  83967.77|
+-----------+-----------+---------+-----------+----------+
only showing top 10 rows



# FIM!